# Compilation Examples with Routing

In [1]:

from pytket import Circuit
from pytket.extensions.aqt.backends.aqt_multi_zone import AQTMultiZoneBackend
from pytket.extensions.aqt.multi_zone_architecture.circuit_routing.settings import (
    RoutingAlg,
    RoutingSettings,
)
from pytket.extensions.aqt.multi_zone_architecture.compilation_settings import (
    CompilationSettings,
)
from pytket.extensions.aqt.multi_zone_architecture.graph_algs.mt_kahypar import (
    MtKahyparConfig,
    configure_mtkahypar,
)
from pytket.extensions.aqt.multi_zone_architecture.initial_placement.settings import (
    InitialPlacementAlg,
    InitialPlacementSettings,
)
from pytket.extensions.aqt.multi_zone_architecture.named_architectures import (
    four_zones_in_a_line,
    grid12,
    grid12_mod,
    racetrack,
    racetrack_4_gatezones,
)

First lets define some circuits

In [2]:
## A simple GHZ circuit
ghz_circuit = Circuit(16)
ghz_circuit.H(0)
for i in range(ghz_circuit.n_qubits - 1):
    ghz_circuit.CX(i, i + 1)
ghz_circuit.measure_all()

## Quantum advantage circuit template
def quantum_advantage_circuit(Lx: int, Ly: int) -> Circuit:
    N = Lx * Ly
    Ntrot = 30
    dt = 0.2
    hx = 2
    J = 1

    def c(x, y):
        if (x - y) % N == Lx or (y - x) % N == Lx:
            return 1
        if x // Lx == y // Lx and x == y - (y % Lx) + (((y % Lx) + 1) % Lx):
            return 1
        if x // Lx == y // Lx and y == x - (x % Lx) + (((x % Lx) + 1) % Lx):
            return 1
        return 0

    coupling_list = []
    for i in range(N):
        for j in range(i + 1, N):
            if c(i, j) == 1:
                coupling_list.append([i, j])

    U = Circuit(N)

    for t in range(Ntrot):
        n_trotter_steps = t

        if n_trotter_steps == 0:
            for j in range(N):
                U.Rx(-2 * dt * hx / 2, j)

        for coupling in coupling_list:
            U.ZZPhase(-2 * dt * J, coupling[0], coupling[1])
        for j in range(N):
            U.Rx(-2 * dt * hx, j)

        if n_trotter_steps == Ntrot - 1:
            for j in range(N):
                U.Rx(2 * dt * hx / 2, j)
    return U

## 56 qubit quantum advantage circuit
advantage_circuit_56 = quantum_advantage_circuit(7, 8)

## 30 qubit quantum advantage circuit
advantage_circuit_30 = quantum_advantage_circuit(5, 6)

def build_qft_circuit(n_qubits: int) -> Circuit:
    circ = Circuit(n_qubits, name="QFT")
    for i in range(n_qubits):
        circ.H(i)
        for j in range(i + 1, n_qubits):
            circ.CU1(1 / 2 ** (j - i), j, i)
    for k in range(n_qubits // 2):
        circ.SWAP(k, n_qubits - k - 1)
    return circ

## 16 qft circuit
qft_circ = build_qft_circuit(16)



Now some MultiZone Backends

In [3]:
line_backend = AQTMultiZoneBackend(
    architecture=four_zones_in_a_line, access_token="invalid"
)

racetrack_backend = AQTMultiZoneBackend(
    architecture=racetrack, access_token="invalid"
)

racetrack_4_gatezones_backend = AQTMultiZoneBackend(
    architecture=racetrack_4_gatezones, access_token="invalid"
)

grid_backend = AQTMultiZoneBackend(
    architecture=grid12, access_token="invalid"
)

grid_mod_backend = AQTMultiZoneBackend(
    architecture=grid12_mod, access_token="invalid"
)


And some different compilation settings

In [4]:
# This can be used to configure the number of threads used and random seed
# for mt-kahypar. It is not required (then default will be used) and can only
# be set once
configure_mtkahypar(
    MtKahyparConfig(
        n_threads = 1,
        random_seed = 13
    )
)
order_init = InitialPlacementSettings(
    algorithm=InitialPlacementAlg.qubit_order,
    zone_free_space=2,
    max_depth=200,
)
graph_routing = RoutingSettings(
    algorithm=RoutingAlg.graph_partition,
    debug_level=1,
)
graph_compilation_settings = CompilationSettings(
    pytket_optimisation_level=1,
    initial_placement=order_init,
    routing=graph_routing,
)

greedy_routing = RoutingSettings(
    algorithm=RoutingAlg.greedy,
)
greedy_compilation_settings = CompilationSettings(
    pytket_optimisation_level=1,
    initial_placement=order_init,
    routing=greedy_routing,
)

## GHZ circuit on a line architecture with two central gate zones and two outer memory zones (8-6-6-8)

In [5]:
ghz_precompiled = line_backend.precompile_circuit(ghz_circuit, graph_compilation_settings)
print("#1qb gates =", ghz_precompiled.n_1qb_gates())
print("#gate depth =", ghz_precompiled.depth())
print("#2qb gates =", ghz_precompiled.n_2qb_gates())
print("#2qb gate depth =", ghz_precompiled.depth_2q())
ghz_graph_routed = line_backend.route_precompiled(ghz_precompiled, graph_compilation_settings, v2=True)
ghz_greedy_routed = line_backend.route_precompiled(ghz_precompiled, greedy_compilation_settings)
print("Shuttles graph: ", ghz_graph_routed.get_n_shuttles())
print("Shuttles greedy: ", ghz_greedy_routed.get_n_shuttles())

#1qb gates = 59
#gate depth = 46
#2qb gates = 15
#2qb gate depth = 15
initial placement {0: [0, 1, 2, 3, 4, 5], 1: [6, 7, 8, 9], 2: [10, 11, 12, 13], 3: [14, 15]}
Depth List:
[(5, 6)]
[(6, 7)]
[(7, 8)]
[(8, 9)]
cut_cost:  2039760
-------
Z0: [0, 1, 2, 3, 4, 5] -> [0, 1, 2, 3, 4, 5, 6] -- (+6)
Z1: [6, 7, 8, 9] -> [7, 8, 9] -- (-6)
Z2: [10, 11, 12, 13] -> [10, 11, 12, 13] -- ()
Z3: [14, 15] -> [14, 15] -- ()
Moving qubit 6 from zone 1 to zone 0
Depth List:
[(6, 7)]
[(7, 8)]
[(8, 9)]
[(9, 10)]
cut_cost:  1993318
-------
Z0: [0, 1, 2, 3, 4, 5, 6] -> [0, 1, 2, 3, 4, 5, 6, 7] -- (+7)
Z1: [7, 8, 9] -> [8, 9] -- (-7)
Z2: [10, 11, 12, 13] -> [10, 11, 12, 13] -- ()
Z3: [14, 15] -> [14, 15] -- ()
Moving qubit 7 from zone 1 to zone 0
Depth List:
[(7, 8)]
[(8, 9)]
[(9, 10)]
[(10, 11)]
cut_cost:  2022644
-------
Z0: [0, 1, 2, 3, 4, 5, 6, 7] -> [0, 1, 2, 3, 4, 5, 7, 8] -- (+8, -6)
Z1: [8, 9] -> [6, 9] -- (+6, -8)
Z2: [10, 11, 12, 13] -> [10, 11, 12, 13] -- ()
Z3: [14, 15] -> [14, 15] -- ()
Moving qub

## QFT circuit on a line architecture with two central gate zones and two outer memory zones (8-6-6-8)

In [7]:
qft_precompiled = line_backend.precompile_circuit(qft_circ, graph_compilation_settings)
print("#1qb gates =", ghz_precompiled.n_1qb_gates())
print("#gate depth =", ghz_precompiled.depth())
print("#2qb gates =", ghz_precompiled.n_2qb_gates())
print("#2qb gate depth =", ghz_precompiled.depth_2q())
ghz_graph_routed = line_backend.route_precompiled(qft_precompiled, graph_compilation_settings, v2=True)
ghz_greedy_routed = line_backend.route_precompiled(qft_precompiled, greedy_compilation_settings)
print("Shuttles graph: ", ghz_graph_routed.get_n_shuttles())
print("Shuttles greedy: ", ghz_greedy_routed.get_n_shuttles())

#1qb gates = 59
#gate depth = 46
#2qb gates = 15
#2qb gate depth = 15
initial placement {0: [0, 1, 2, 3, 4, 5], 1: [6, 7, 8, 9], 2: [10, 11, 12, 13], 3: [14, 15]}
Depth List:
[(0, 6), (0, 6)]
[(1, 6), (0, 7), (1, 6), (0, 7)]
[(2, 6), (1, 7), (2, 6), (0, 8), (1, 7), (0, 8)]
[(3, 6), (2, 7), (3, 6), (1, 8), (2, 7), (0, 9), (1, 8), (0, 9)]
cut_cost:  2089380
-------
Z0: [0, 1, 2, 3, 4, 5] -> [0, 1, 2, 3, 4, 5, 6, 7] -- (+6, +7)
Z1: [6, 7, 8, 9] -> [8, 9] -- (-6, -7)
Z2: [10, 11, 12, 13] -> [10, 11, 12, 13] -- ()
Z3: [14, 15] -> [14, 15] -- ()
Moving qubit 7 from zone 1 to zone 0
Moving qubit 6 from zone 1 to zone 0
Depth List:
[(0, 8), (0, 8)]
[(1, 8), (0, 9), (1, 8), (0, 9)]
[(2, 8), (1, 9), (2, 8), (0, 10), (1, 9), (0, 10)]
[(3, 8), (2, 9), (3, 8), (1, 10), (2, 9), (0, 11), (1, 10), (0, 11)]
cut_cost:  2077406
-------
Z0: [0, 1, 2, 3, 4, 5, 6, 7] -> [0, 1, 2, 3, 4, 5, 7, 8] -- (+8, -6)
Z1: [8, 9] -> [6, 9] -- (+6, -8)
Z2: [10, 11, 12, 13] -> [10, 11, 12, 13] -- ()
Z3: [14, 15] -> [14, 1

## 56 qubit "quantum advantage" circuit on all gate-zone "racetrack" architecture

In [11]:

adv_precomp = racetrack_backend.precompile_circuit(advantage_circuit_56, graph_compilation_settings)
print("#1qb gates =", adv_precomp.n_1qb_gates())
print("#gate depth =", adv_precomp.depth())
print("#2qb gates =", adv_precomp.n_2qb_gates())
print("#2qb gate depth =", adv_precomp.depth_2q())
adv_graph_routed = racetrack_backend.route_precompiled(adv_precomp, graph_compilation_settings, v2=True)
adv_greedy_routed = racetrack_backend.route_precompiled(adv_precomp, greedy_compilation_settings)
print("Shuttles graph: ", adv_graph_routed.get_n_shuttles())
print("Shuttles greedy: ", adv_greedy_routed.get_n_shuttles())

#1qb gates = 8700
#gate depth = 2162
#2qb gates = 6720
#2qb gate depth = 1094
initial placement {0: [0, 1, 2, 3], 1: [4, 5, 6, 7], 2: [8, 9, 10, 11], 3: [12, 13, 14, 15], 4: [16, 17, 18, 19], 5: [20, 21, 22, 23], 6: [24, 25, 26, 27], 7: [28, 29, 30, 31], 8: [32, 33, 34, 35], 9: [36, 37, 38, 39], 10: [40, 41, 42, 43], 11: [44, 45, 46, 47], 12: [48, 49, 50, 51], 13: [52, 53, 54, 55], 14: [], 15: [], 16: [], 17: [], 18: [], 19: [], 20: [], 21: [], 22: [], 23: [], 24: [], 25: [], 26: [], 27: []}
Depth List:
[(0, 6), (0, 6), (1, 8), (1, 8), (2, 9), (3, 4), (2, 9), (3, 4)]
[(0, 7), (0, 7), (1, 50), (1, 50), (2, 51), (3, 10), (2, 51), (4, 5), (3, 10), (4, 5)]
[(0, 49), (7, 8), (0, 49), (7, 8), (3, 52), (4, 11), (3, 52), (5, 6), (4, 11), (5, 6)]
[(8, 9), (0, 1), (8, 9), (0, 1), (4, 53), (5, 12), (4, 53), (6, 13), (5, 12), (6, 13)]
cut_cost:  12660830
current config {0: [0, 1, 2, 3], 1: [4, 5, 6, 7], 2: [8, 9, 10, 11], 3: [12, 13, 14, 15], 4: [16, 17, 18, 19], 5: [20, 21, 22, 23], 6: [24, 25, 2

## 56 qubit "quantum advantage" circuit on 4 sequential gate-zone "racetrack" architecture

In [8]:

adv_precomp2 = racetrack_4_gatezones_backend.precompile_circuit(advantage_circuit_56, graph_compilation_settings)
print("#1qb gates =", adv_precomp2.n_1qb_gates())
print("#gate depth =", adv_precomp2.depth())
print("#2qb gates =", adv_precomp2.n_2qb_gates())
print("#2qb gate depth =", adv_precomp2.depth_2q())
adv_graph_routed2 = racetrack_4_gatezones_backend.route_precompiled(adv_precomp2, graph_compilation_settings)
adv_greedy_routed2 = racetrack_4_gatezones_backend.route_precompiled(adv_precomp2, greedy_compilation_settings)
print("Shuttles graph: ", adv_graph_routed2.get_n_shuttles())
print("Shuttles greedy: ", adv_greedy_routed2.get_n_shuttles())

#1qb gates = 8700
#gate depth = 2162
#2qb gates = 6720
#2qb gate depth = 1094
Shuttles graph:  11286
Shuttles greedy:  18826


## Grid Architecture: All gate zones

In [9]:

adv_precomp = grid_backend.precompile_circuit(advantage_circuit_30, graph_compilation_settings)
print("#1qb gates =", adv_precomp.n_1qb_gates())
print("#gate depth =", adv_precomp.depth())
print("#2qb gates =", adv_precomp.n_2qb_gates())
print("#2qb gate depth =", adv_precomp.depth_2q())
adv_graph_routed = grid_backend.route_precompiled(adv_precomp, graph_compilation_settings)
adv_greedy_routed = grid_backend.route_precompiled(adv_precomp, greedy_compilation_settings)
print("Shuttles graph: ", adv_graph_routed.get_n_shuttles())
print("Shuttles greedy: ", adv_greedy_routed.get_n_shuttles())

#1qb gates = 4654
#gate depth = 1269
#2qb gates = 3600
#2qb gate depth = 640
Shuttles graph:  1837
Shuttles greedy:  2550


## Grid Architecture: Four gate zones

In [10]:
adv_gm_precompiled = grid_mod_backend.precompile_circuit(advantage_circuit_30, graph_compilation_settings)
print("#1qb gates =", adv_gm_precompiled.n_1qb_gates())
print("#gate depth =", adv_gm_precompiled.depth())
print("#2qb gates =", adv_gm_precompiled.n_2qb_gates())
print("#2qb gate depth =", adv_gm_precompiled.depth_2q())
adv_gm_graph_routed = grid_mod_backend.route_precompiled(adv_gm_precompiled, graph_compilation_settings)
adv_gm_greedy_routed = grid_mod_backend.route_precompiled(adv_gm_precompiled, greedy_compilation_settings)
print("Shuttles graph: ", adv_gm_graph_routed.get_n_shuttles())
print("Shuttles greedy: ", adv_gm_greedy_routed.get_n_shuttles())

#1qb gates = 4654
#gate depth = 1269
#2qb gates = 3600
#2qb gate depth = 640
Shuttles graph:  3371
Shuttles greedy:  3633
